In [2]:
import cv2
import mediapipe.python.solutions.hands as mp_hands
import mediapipe.python.solutions.drawing_utils as mp_drawing
from pynput.keyboard import Controller, Key

keyboard = Controller()

# ----------------- ฟังก์ชันช่วยเรื่องนิ้ว ----------------- #
def get_finger_states(hand_landmarks, margin=0.005):
    lm = hand_landmarks.landmark # ลิสต์ 21 จุด

    finger_tips = { 
        "index": mp_hands.HandLandmark.INDEX_FINGER_TIP.value,
        "middle": mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value,
        "ring": mp_hands.HandLandmark.RING_FINGER_TIP.value,
        "pinky": mp_hands.HandLandmark.PINKY_TIP.value,
    }

    finger_pips = { 
        "index": mp_hands.HandLandmark.INDEX_FINGER_PIP.value,
        "middle": mp_hands.HandLandmark.MIDDLE_FINGER_PIP.value,
        "ring": mp_hands.HandLandmark.RING_FINGER_PIP.value,
        "pinky": mp_hands.HandLandmark.PINKY_PIP.value,
    }

    fingers_extended = {} 
    for name in finger_tips.keys():
        tip = lm[finger_tips[name]]
        pip = lm[finger_pips[name]]
        fingers_extended[name] = tip.y < (pip.y - margin)

    return fingers_extended

def classify_hand_gesture(hand_landmarks): # แปลท่ามือ → LEFT / RIGHT / JUMP / NONE
    lm = hand_landmarks.landmark
    fingers = get_finger_states(hand_landmarks)

    # ดึงจุดของนิ้วชี้
    index_tip = lm[mp_hands.HandLandmark.INDEX_FINGER_TIP.value]
    index_mcp = lm[mp_hands.HandLandmark.INDEX_FINGER_MCP.value]

    # dx > 0 → ชี้ไปทางขวา, dx < 0 → ชี้ไปทางซ้าย
    # dy < 0 → ชี้ขึ้น, dy > 0 → ชี้ลง
    dx = index_mcp.x - index_tip.x
    dy = index_tip.y - index_mcp.y

    if not (fingers["index"] and not (fingers["middle"] or fingers["ring"] or fingers["pinky"])):
        return "NONE"

    if abs(dx) > abs(dy) * 1.2 and abs(dx) > 0.03:
        if dx > 0:
            return "RIGHT"
        else:
            return "LEFT"
    if dy < -0.01:
        return "JUMP"

    return "NONE"

key_states = {}

def set_key_state(key, should_be_pressed):
    prev = key_states.get(key, False)
    if should_be_pressed and not prev:
        keyboard.press(key)
        key_states[key] = True
    elif not should_be_pressed and prev:
        keyboard.release(key)
        key_states[key] = False




# ----------------- ฟังก์ชันหลัก ควบคุมเกมด้วยมือทั้งหมด + เปิดกล้องเว็บแคม ----------------- #
def run_hand_control():
    cap = cv2.VideoCapture(0)

    with mp_hands.Hands(
        model_complexity=0,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
    ) as hands:

        while cap.isOpened(): # รัน loop เมื่อกล้องเปิดอยู่
            success, frame = cap.read()
            if not success:
                print("Ignoring empty camera frame...")
                continue

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)

            # เตรียมโครงสร้างปุ่มเดินที่ควรกด 
            desired_pressed = {
                'a': False,        
                'd': False,        
                Key.left: False,   
                Key.right: False,  
            }

            debug_lines = []
            # ตรวจดูว่าเจอกี่มือ และดูว่าเป็นมือซ้ายหรือขวา
            if results.multi_hand_landmarks and results.multi_handedness:
                for hand_landmarks, handedness in zip(
                    results.multi_hand_landmarks,
                    results.multi_handedness
                ):
                    label = handedness.classification[0].label   # "Left" / "Right"
                    gesture = classify_hand_gesture(hand_landmarks)

                    # วาดมือ
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                    )

                    debug_lines.append(f"{gesture}")

                    # ----------------- mapping มือซ้าย/ขวา ----------------- #
                    if label == "Right":
                        if gesture == "LEFT":
                            desired_pressed['a'] = True
                        elif gesture == "RIGHT":
                            desired_pressed['d'] = True
                        elif gesture == "JUMP":
                            # ถ้าตอนนี้กำลังเดินอยู่แล้ว ให้เดินต่อไปขณะกระโดด
                            if key_states.get('a', False):
                                desired_pressed['a'] = True
                            if key_states.get('d', False):
                                desired_pressed['d'] = True

                            # กระโดด
                            keyboard.press('w')
                            keyboard.release('w')

                    elif label == "Left":
                        # Fireboy
                        if gesture == "LEFT":
                            desired_pressed[Key.left] = True
                        elif gesture == "RIGHT":
                            desired_pressed[Key.right] = True
                        elif gesture == "JUMP":
                            # ถ้าตอนนี้กำลังเดินอยู่แล้ว ให้เดินต่อไปขณะกระโดด
                            if key_states.get(Key.left, False):
                                desired_pressed[Key.left] = True
                            if key_states.get(Key.right, False):
                                desired_pressed[Key.right] = True

                            # กระโดด
                            keyboard.press(Key.up)
                            keyboard.release(Key.up)

            # ---- วนทุกKey และส่งให้กด ---- #
            for key, should_press in desired_pressed.items():
                set_key_state(key, should_press)

            # ---- flip & render ---- #
            flipped = cv2.flip(frame, 1)

            for i, line in enumerate(debug_lines):
                cv2.putText(
                    flipped,
                    line,
                    (10, 30 + 30 * i),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8,
                    (0, 255, 0),
                    2,
                    cv2.LINE_AA,
                )

            cv2.imshow("Hand Control", flipped)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    run_hand_control()
